# Week3_2 Assignment

## [BASIC](#Basic) 
- 한국어 코퍼스를 로드해 **WordPiece Tokenzier를 학습**시킬 수 있다.
- 학습된 모델을 로드해 **encoding과 decoding을 수행**할 수 있다. 



### Reference
- [BertWordPieceTokenizer 학습 소개 한국어 블로그](https://monologg.kr/2020/04/27/wordpiece-vocab/)
- [huggingface python train tutorial](https://github.com/huggingface/tokenizers/blob/master/bindings/python/examples/train_bert_wordpiece.py)

In [ ]:
# colab GPU 제한때문에 train 시간이 모자라 실행 결과를 못 남긴채로 제출합니다.

import os 
import sys
import pandas as pd
import numpy as np

In [ ]:
!pip install tokenizers

In [ ]:
import torch
from tokenizers import BertWordPieceTokenizer

In [ ]:
# seed
seed = 7777
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [ ]:
# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

cpu


## Basic

### 데이터 다운로드
- 내 구글 드라이브에 데이터를 다운 받은 후 코랩에 드라이브를 마운트하면 데이터를 영구적으로 사용할 수 있다. 
- [데이터 다운로드 출처](https://ratsgo.github.io/embedding/downloaddata.html)

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
cd '/content/drive/MyDrive/Wiki_corpus'

/content/drive/MyDrive/Wiki_corpus


In [ ]:
# 한국어 위키피디아 데이터 (토크나이즈되지 않은 텍스트) 로드
!pip install gdown
!gdown https://drive.google.com/u/0/uc?id=1kUecR7xO7bsHFmUI6AExtY5u2XXlObOG
!unzip processed.zip

Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/u/0/uc?id=1kUecR7xO7bsHFmUI6AExtY5u2XXlObOG 

unzip:  cannot find or open processed.zip, processed.zip.zip or processed.zip.ZIP.


In [ ]:
_CUR_DIR = os.path.abspath(os.curdir)
print(f"My current directory : {_CUR_DIR}")
_DATA_DIR = os.path.join(_CUR_DIR, "processed")

My current directory : /content/drive/MyDrive/Wiki_corpus


### 한국어 위키피디아 코퍼스로 WordPiece tokenizer 학습
- 한국어 위키 

In [ ]:
# processed_wiki_ko.txt 파일 불러오기
with open('processed_wiki_ko.txt') as f:
  docs = f.readlines()

In [ ]:
print(f"# wiki documents: {len(docs):,}")

# wiki documents: 311,237


In [ ]:
# Word Piece Tokenizer 인스턴스 생성
tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=True,
    strip_accents=True, # 악센트가 있는 character의 악센트를 제거하려면? (ex. é → e)
    lowercase=False # 한국어는 대소문자가 없는데 소문자 변환이 필요한지?
)

In [ ]:
# train
# files: 'processed_wiki_ko.txt'
# vocab_size: 30,000
# min_frequency: 2
# special_tokens = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"]
# limit_alphabet: 1,000
# wordpieces_prefix: '##'

tokenizer.train(
    files = 'processed_wiki_ko.txt',
    vocab_size = 30000,
    min_frequency = 2,
    show_progress = True,
    special_tokens = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
    limit_alphabet = 1000,
    wordpieces_prefix = '##',
)

In [ ]:
tokenizer.save_model(".", "wordpiece")

### Encoding
- 저장된 토크나이즈 파일을 로드해 `BertWordPieceTokenizer` 인스턴스를 생성하고 다음을 수행하자. 
    - 사전(vocab)의 단어 개수를 출력
    - 문장을 토크나이징한 후 토큰 id와 토큰 string을 출력

In [ ]:
tokenizer = BertWordPieceTokenizer(
    vocab = "wordpiece-vocab.txt 경로",
    lowercase = False,
    strip_accents = False,
)

In [ ]:
# 사전 단어 개수 출력
tokenizer.get_vocab_size()

In [ ]:
text = "안녕하세요. 버트를 사용한 모델입니다."

# 토크나이즈한 후 토큰의 id를 출력하라 
token_ids = tokenizer.encode(text, add_special_tokens=False)
print(token_ids)

# 토크나이즈한 후 각 토큰(string)을 출력하라.
tokens = tokenizer.encode(text, add_special_tokens=True)
print(tokens)

In [ ]:
# 신조어를 토크나이징할 수 있는지 테스트해보자.
text = "신조어 입력"
tokens = tokenizer.encode(text, add_special_tokens=True)
print(tokens)

In [ ]:
# 사전에 없는 단어는 어떻게 토크나이즈 되는가?
text = "사전에 없는 단어 입력"
unknown_token_ids = tokenizer.encode(text, add_special_tokens=False) # 토큰 id
unknown_tokens = tokenizer.encode(text, add_special_tokens=True) # 토큰
print(unknown_token_ids)
print(unknown_tokens)

### Decoding
- 토큰 id를 원래 문장으로 디코딩하자.

In [ ]:
# 원래 문장: "안녕하세요. 버트를 사용한 모델입니다."
tokenizer.decode(token_ids)

In [ ]:
# 사전에 없는 단어는 어떻게 디코딩되는가?
tokenizer.decode(unknown_token_ids)